# 3. __Classical Controllers__

- The control structure of power electronic inverters can be divided into cascading levels. 
- The second level above the current control loop is a voltage control loop.
- A third level on top of the voltage control is called "Droop".

Classically controlled modes:

|  | Mode | Description |
| --- | --- | --- |
| 1 | "Swing" | Ideal voltage source without dynamics (i.e. an Infinite Bus) |
| 2 | "PQ" | Grid following controllable source/load (real and imaginary power) |
| 3 | __"Droop"__ | __Simple grid forming with power balancing through a droop mechanism__ |
| 4 | "Synchronverter" or "VSG"|  Grid forming control mimicking a generator, i.e. Virtual Synchronous Generator |


## Droop Mode - Simple Grid Forming

- This example is intended to introduce you to a decentralised and self-organising control mode.
- The mode "communicates" with other inverters in the network through the voltage and frequency.
- Battery Energy Storage Systems (BESSs) can be represented by this mode.
- This mode is autonomous and provides a measure of "inertia" to the network.

### Theory

The droop control method can be referred to as an independent, autonomous, and wireless control since intercommunication links between the converters other than the electrical cables can be eliminated. That is, in a network with loads and inverters in Droop mode, the power required by the loads is partitioned amongst the inverters in Droop modes by only varying electrical quantities are their respective control points. 

This mode works quite well when high frequency harmonics and switching ripples are neglected (as they presently are in the set-up). Unlike the PQ mode, a source in Droop mode has to have some energy storage available. 

The voltage control loop has a very similar structure to the inner current control loop, also operating in the DQ0 frame and with PI controllers. The "Droop" control sitting on top of the voltage control loop, provides the reference voltage signals. 
<br><br>
<div>
<img src="figures/voltage_control.jpg" width="500"/>
</div>

The droop mode for every inverter has two coefficients, Dp and Dq, that determine its behaviour. Dp is associated with the real power and frequency, while Dq corresponds to the voltage magnitude at the Point-of-Control (POC) and the imaginary power. These coefficients can by tuned either heuristically or via more advanced algorithms like particle swarm optimisation.

<br>
<div>
<img src="figures/Droop_control.jpg" width="500"/>
</div>

If the user does not specify Dp and Dq, then they are automatically tuned based on the converter power rating and the network's maximum allowable frequency and voltage deviations, i.e. Δfmax and ΔEmax. 

$$
  D_p = \frac{S_{rated}}{{ω_{set}}^2 Δfmax}
$$

$$
  D_q = \frac{S_{rated}}{V_{set}*\sqrt{2}*ΔEmax}
$$

<br>
<div>
<img src="figures/Droop_mode.jpg" width="700"/>
</div>

In [1]:
using JEG;

WebIO._IJuliaInit()

_______________________________________________________________________________
### Network Configuration 

>- Two inverters are connected to each other through a cable.
>- The one inverter is placed in the familiar PQ mode, while the other is in Droop mode.
>- The Droop inverter will regulate the frequency and the PQ inverter will follow along.
>- If not provided by the user, the proportional and integral gains for the PI controllers are tuned automatically via a loop-shaping method.

In [2]:
# total run time, seconds
t_end = 0.08     

# Connectivity Matrix
CM = [ 0. 1.
        -1. 0.]     

parameters = Dict{Any, Any}(
        "source" => Any[
                        Dict{Any, Any}("pwr"    => 400e3,   # Rated appared power [VA] 
                                        "mode"  => "Droop", # Controller mode
                                        "V_kp"  => 0.6,     # Voltage proportional gain [A/V] (**optional**)
                                        "V_ki"  => 12,      # Current integral gain [A/V⋅s] (**optional**)
                                        "Dp"    => 810,     # Frequency droop coefficient [N⋅m⋅s/rad] (**optional**)
                                        "Dq"    => 24e3)    # Voltage droop coefficient [VAi/V] (**optional**)
                        Dict{Any, Any}("pwr"    => 100e3,   
                                        "mode"  => "PQ",    
                                        "p_set" => 50e3,    # Real power set point (generating) [W] 
                                        "q_set" => -40e3)   # Imaginary power set point (inductive) [VAi]  
                        ],
        "cable"   => Any[
                        Dict{Any, Any}("R"  => 0.1, 
                                        "L" => 0.25e-3, 
                                        "C" => 0.1e-4),
                        ],
        "grid"   => Dict{Any, Any}("Δfmax"  => 0.5, # The % drop (increase) in frequency that causes a 100% increase (decrease) in active power (from nominal).
                                    "ΔEmax" => 5)   # The % drop (increase) in voltage that causes a 100% increase (decrease) in reactive power (from nominal).

    );

env = ElectricGridEnv(CM = CM, parameters = parameters, t_end = t_end, verbosity = 2);

agents = SetupAgents(env);

hook = Simulate(agents, env);

[ Info: Normalization is done based on the defined parameter limits.
[ Info: Time simulation run time: 0.08 [s] ~> 801 steps
[ Info: 2 'classically' controlled sources have been initialised.
[ Info: 1 source has been set up in PQ mode.
[ Info: 1 source has been set up in Droop mode.
[ Info: 2 sources have automatically calculated proportional and integral gains for their current control loops.


_______________________________________________________________________________
### Rendering

In [3]:
RenderHookResults(hook = hook, 
                    states_to_plot  = [], 
                    actions_to_plot = [],  
                    v_mag_poc       = [1],   # Scaled L₂ norm in αβγ coordinates [V]
                    power_p_inv     = [2],   # Real power [Watts]
                    power_q_inv     = [2],   # Imaginary power [VAi]
                    power_p_poc     = [1],   # Real power [Watts]
                    power_q_poc     = [1],   # Imaginary power [VAi]
                    angles          = [1 2], # Relative angle [degrees]
                    freq            = [1 2], # Angular velocity [Hz]
                    )

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields height, legend, margin, plot_bgcolor, template, width, xaxis, yaxis, and yaxis2"

_______________________________________________________________________________
### Analysis

- As expected, the inverter in PQ mode behaves as it ideally should.
- Notice that the frequency of the network is not exactly 50Hz. This is due to the frequency droop mechanism.
- Neither does the voltage magnitude in steady state equal 230 V. This is due to the voltage droop mechanism.